In [ ]:
import requests
import json
import os

In [ ]:
api_key = os.environ["YELP_API_KEY"]
headers = {'Authorization': 'Bearer %s' % api_key}

In [ ]:
url='https://api.yelp.com/v3/businesses/search'
parameters = [{'term':'Indian Restaurants','location':'Manhattan', 'limit': 50, 'offset': 0}, 
             {'term':'Chinese Restaurants','location':'Manhattan','limit': 50, 'offset': 0}, 
              {'term':'Italian Restaurants','location':'Manhattan','limit': 50, 'offset': 0},
             {'term':'Korean Restaurants','location':'Manhattan','limit': 50, 'offset': 0}, 
             {'term':'American Restaurants','location':'Manhattan','limit': 50, 'offset': 0},
             {'term':'Mexican Restaurants','location':'Manhattan','limit': 50, 'offset': 0}]


In [ ]:
data={'Indian':[], 'Chinese':[],'Italian':[],'Korean':[],'American':[],'Mexican':[]}
for params in parameters:
    name, _ = params['term'].split(" ")
    rest_data = []
    while(params['offset']<1000):
        req=requests.get(url, params=params, headers=headers)
        #print(params, req.status_code)
        if(req.status_code==200):
            #json.loads(req.text)
            dic = json.loads(req.text)
            d = dic['businesses']
            rest_data.extend(d)
            data[name].extend(d)
            params['offset']+=50

#json.dump(unique_data, outfile)

In [ ]:
for name in data:
    data[name] = list({ each["id"] : each for each in data[name]}.values())

In [ ]:
print(len(data))

In [ ]:
import boto3

In [ ]:
dynamodb = boto3.resource('dynamodb')

In [ ]:
table = dynamodb.create_table(
    TableName='yelp-restaurants',
    KeySchema=[
        {
            'AttributeName': 'business_id',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'insertedAtTimestamp',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'business_id',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'insertedAtTimestamp',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)
table.meta.client.get_waiter('table_exists').wait(TableName='yelp-restaurants')
print(table.item_count)

In [ ]:
from datetime import datetime
from decimal import Decimal

def empty_check(string):
    if str(string)=="":
        return "na"
    else:
        return str(string)

restaurants = data
hash_rest_id = dict()
with table.batch_writer() as batch:
    for types in restaurants:
        rest_types = restaurants[types]
        for rest in rest_types:
            #print (rest)
            business_id = rest['id']
            if business_id in hash_rest_id:
                continue
            hash_rest_id[business_id]=1
            name = empty_check(rest['name'])
            address = empty_check(rest['location']['display_address'])
            coordinates = empty_check(rest['coordinates'])
            no_of_reviews = empty_check(rest['review_count'])
            rating = empty_check(rest['rating'])
            zip_code = empty_check(rest['location']['zip_code'])
            
            batch.put_item(
                Item={'business_id':str(business_id),
                      'insertedAtTimestamp':datetime.utcnow().isoformat(),
                      'cuisine':str(types),
                      'name':str(name),
                      'address':str(address),
                      'coordinates':str(coordinates),
                      'no_of_reviews':str(no_of_reviews),
                      'rating':str(rating),
                      'zip_code':str(zip_code),
                }
            )

In [ ]:
#table = dynamodb.Table('yelp-restaurants')
dynamoDBClient = boto3.client('dynamodb')
table = dynamoDBClient.describe_table(
    TableName='yelp-restaurants'
)
print(table)